In [1]:
#这个是将多帧dicom中所有的图片进行提取
!pip install pydicom

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 674.6 kB/s eta 0:00:0000:0100:01


In [10]:
import pydicom
import math
import os
import sys
import time
import json


def generateImagesFromDicom(dataset, filename="extracted_folder", extractfps=0):
    """
        Go through all items in the dataset and print them with custom format

        Modelled after Dataset._pretty_str()
    """
    cine_rate = 12    
    for data_element in dataset:
        if data_element.name == 'Cine Rate':
            print(f"Cine Rate:{data_element.value}, {type(data_element.value)}")
            cine_rate = data_element.value.real
        if data_element.name == 'Pixel Data':
            pixel_datas = data_element.value
       
    # check frame number and tranfersyntex here
    frmpath = filename + '_frms'
    os.makedirs(filename + '_frms', exist_ok=True)
    
    print('Extract frames from [Pixel Data]:')
    if 0 == extractfps or extractfps > cine_rate:
        delta_step = cine_rate / cine_rate
    else:
        delta_step = cine_rate / extractfps
    frmindex = 0
    for sequence_index, frm in enumerate(pydicom.encaps.generate_pixel_data_frame(pixel_datas)):
        delta = sequence_index - (frmindex * delta_step)
        if delta >= 1 or delta < 0:
            continue
        frmname = os.path.join(frmpath, f"frm-{(frmindex+1):04d}.png")
        #print("generateImagesFromDicom: {0} {1} : {2} bytes".format(">>", frmname, len(frm)))
        with open(frmname, 'wb+') as f:
            f.write(frm)
        frmindex += 1
    print(f"Total generate {frmindex} image...")

    return frmpath if frmindex > 1 else None

def record2File(imagedir, fps, raw_file_type='mp4'):
    """when finish the extract action, make a file to record the status"""
    jsonfile = "lmstart.json"
    now = time.localtime(time.time())
    datatime = "{0}-{1}-{2} {3}:{4}:{5}".format(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour,now.tm_min, now.tm_sec)
    recorder = {"datetime":datatime, "fps":fps, "imagepath":imagedir, "raw_file_type": raw_file_type}
    print(recorder)
    
    with open(os.path.join(imagedir, jsonfile), "w") as fp:
        json.dump(recorder, fp, ensure_ascii=True)
    return 0
    
def extractDcmToImages(dicom_file, extractfps):
    print("dcmfile:", dicom_file)
    if os.path.isfile(dicom_file) and  os.path.splitext(dicom_file)[1] ==".dcm":
        print("dcmfile is file? :", dicom_file)
        ds = pydicom.dcmread(dicom_file)
        try:
            image_dir = generateImagesFromDicom(ds, filename=dicom_file, extractfps=extractfps)
        except Exception as e:
            print('Err:dcm library error')
            image_dir = None
        if image_dir:
            record2File(imagedir=image_dir, fps=extractfps, raw_file_type="dcm")
        return image_dir
    else:
        return None

In [11]:
def extract_all_dcm_imgs(dcm_root_dir, count=0):
    if not os.path.exists(dcm_root_dir):
        print(f"Dir not exist:{dcm_root_dir}")
        return -1
    subdirs = []
    dirs = sorted(os.listdir(dcm_root_dir))
    print(f"dirs={dirs}")
    for item in dirs:
        path = os.path.join(dcm_root_dir, item)
        if os.path.isdir(path):
            subdirs.append(path)
    if len(subdirs) <1:
        subdirs.append(dcm_root_dir)
    print(f"subdirs={subdirs}")
    
    folder_details = []
    dcm_f_count = 0
    for ct, dcm_dir in enumerate(subdirs):
        print("*"*32)
        print('Handle with {0} [{1}/]'.format(dcm_dir, ct + 1))
        files_in_dir = os.listdir(dcm_dir)
        if len(files_in_dir) < 2:
            getLogger().info(f"Empty dirctory : {dcm_dir}")
            continue
        else:
            for dcmf in files_in_dir:
                fullfilename = os.path.join(dcm_dir, dcmf)
                if not os.path.isfile(fullfilename):
                    continue
                print("file in dir:", fullfilename)
                extractDcmToImages(fullfilename, 100)
                dcm_f_count = dcm_f_count+1
                if count != 0 and count == dcm_f_count:
                    print(f"\n<<<already extract {count} directory dcm file~")
                    return
    return 0

In [7]:
## worked at 21/08/22
#dcm_root_dir=r"/mnt/d/workspace/01-videos"
#extract_all_dcm_imgs(dcm_root_dir, 1)

In [5]:
##working at 22/01/29
dcm_root_dir=r"/mnt/o/download/220222_thyroids"
extract_all_dcm_imgs(dcm_root_dir, 0)

Dir not exist:/mnt/o/download/220222_thyroids


-1

In [6]:
#241130 extract dcm mini batch for new annotation files
dcm_root_dir="/mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch"
extract_all_dcm_imgs(dcm_root_dir, 0)

dirs=['1.2.250.1.204.5.667136.20211130072913172137.2.0.50.80.2.20241025120951138_MARK.json', '甲结节1.dcm']
subdirs=['/mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch']
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch [1/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch/1.2.250.1.204.5.667136.20211130072913172137.2.0.50.80.2.20241025120951138_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch/1.2.250.1.204.5.667136.20211130072913172137.2.0.50.80.2.20241025120951138_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch/甲结节1.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch/甲结节1.dcm
dcmfile is file? : /mnt/f/240926-RayShap/241129-thyroid-datas/01-mini-batch/甲结节1.dcm
Cine Rate:56, <class 'pydicom.valuerep.IS'>
Extract frames from [Pixel Data]:
Total generate 423 image...
{'datetime': '2024-11-30 16:13:49', 'fps': 100, 'imagepath': '/mnt/f/2

0

In [7]:
#241130 extract dcm mini batch for new annotation files
dcm_root_dir="/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt"
extract_all_dcm_imgs(dcm_root_dir, 0)

dirs=['renameDcmFiles.sh', '甲结节10_甲状腺结节标注_标注_审核_仲裁', '甲结节11_甲状腺结节标注_标注_审核_仲裁_质控', '甲结节12_甲状腺结节标注_标注_审核_仲裁', '甲结节1_甲状腺结节标注_标注_审核_仲裁', '甲结节2_甲状腺结节标注_标注_审核_仲裁_质控', '甲结节3_甲状腺结节标注_标注_审核_仲裁', '甲结节5_甲状腺结节标注_标注_审核_仲裁', '甲结节6_甲状腺结节标注_标注_审核_仲裁_质控', '甲结节7_甲状腺结节标注_标注_审核_仲裁', '甲结节9_甲状腺结节标注_标注_审核_仲裁']
subdirs=['/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节10_甲状腺结节标注_标注_审核_仲裁', '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节11_甲状腺结节标注_标注_审核_仲裁_质控', '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节12_甲状腺结节标注_标注_审核_仲裁', '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节1_甲状腺结节标注_标注_审核_仲裁', '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节2_甲状腺结节标注_标注_审核_仲裁_质控', '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节3_甲状腺结节标注_标注_审核_仲裁', '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节5_甲状腺结节标注_标注_审核_仲裁', 

Extract frames from [Pixel Data]:
Total generate 339 image...
{'datetime': '2024-11-30 18:35:45', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节3_甲状腺结节标注_标注_审核_仲裁/thyroidNodules_axp003.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节5_甲状腺结节标注_标注_审核_仲裁 [7/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节5_甲状腺结节标注_标注_审核_仲裁/1.2.250.1.204.5.667136.20211130073046160577.2.0.50.80.2.20241025154334222_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节5_甲状腺结节标注_标注_审核_仲裁/1.2.250.1.204.5.667136.20211130073046160577.2.0.50.80.2.20241025154334222_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/甲结节5_甲状腺结节标注_标注_审核_仲裁/thyroidNodules_axp005.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/02-batch

0

In [9]:
extractDcmToImages(r'/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/thyroidNodules_axp-009/thyroidNodules_axp009.dcm', 100)

dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/thyroidNodules_axp-009/thyroidNodules_axp009.dcm
dcmfile is file? : /mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/thyroidNodules_axp-009/thyroidNodules_axp009.dcm
Cine Rate:51, <class 'pydicom.valuerep.IS'>
Extract frames from [Pixel Data]:
Total generate 206 image...
{'datetime': '2024-11-30 23:4:55', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/thyroidNodules_axp-009/thyroidNodules_axp009.dcm_frms', 'raw_file_type': 'dcm'}


'/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt/thyroidNodules_axp-009/thyroidNodules_axp009.dcm_frms'

In [12]:
#241201 extract dcm rest72 batch for new annotation files
dcm_root_dir="/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat"
extract_all_dcm_imgs(dcm_root_dir, 0)

dirs=['renameDcmFiles.sh', 'renameFolderToEnglist.sh', 'thyroidNodules_axp013', 'thyroidNodules_axp014', 'thyroidNodules_axp015', 'thyroidNodules_axp016', 'thyroidNodules_axp017', 'thyroidNodules_axp018', 'thyroidNodules_axp019', 'thyroidNodules_axp020', 'thyroidNodules_axp021', 'thyroidNodules_axp022', 'thyroidNodules_axp023', 'thyroidNodules_axp024', 'thyroidNodules_axp025', 'thyroidNodules_axp026', 'thyroidNodules_axp027', 'thyroidNodules_axp028', 'thyroidNodules_axp029', 'thyroidNodules_axp030', 'thyroidNodules_axp031', 'thyroidNodules_axp032', 'thyroidNodules_axp033', 'thyroidNodules_axp034', 'thyroidNodules_axp035', 'thyroidNodules_axp036', 'thyroidNodules_axp037', 'thyroidNodules_axp038', 'thyroidNodules_axp040', 'thyroidNodules_axp041', 'thyroidNodules_axp042', 'thyroidNodules_axp043', 'thyroidNodules_axp044', 'thyroidNodules_axp045', 'thyroidNodules_axp046', 'thyroidNodules_axp047', 'thyroidNodules_axp048', 'thyroidNodules_axp049', 'thyroidNodules_axp050', 'thyroidNodules_axp0

Total generate 281 image...
{'datetime': '2024-12-1 21:33:34', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp013/thyroidNodules_axp-013.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp014 [2/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp014/1.2.250.1.204.5.667136.20211130081914852518.2.0.50.80.2.20241025193206142_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp014/1.2.250.1.204.5.667136.20211130081914852518.2.0.50.80.2.20241025193206142_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp014/thyroidNodules_axp-014.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyr

Total generate 323 image...
{'datetime': '2024-12-1 21:33:45', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp020/thyroidNodules_axp-020.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp021 [9/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp021/1.2.250.1.204.5.667136.20210924063824727961.2.0.50.80.2.20241026075314466_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp021/1.2.250.1.204.5.667136.20210924063824727961.2.0.50.80.2.20241026075314466_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp021/thyroidNodules_axp-021.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyr

Total generate 242 image...
{'datetime': '2024-12-1 21:33:52', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp027/thyroidNodules_axp-027.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp028 [16/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp028/1.2.250.1.204.5.667136.20211123074232645172.2.0.50.80.2.20241026132721115_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp028/1.2.250.1.204.5.667136.20211123074232645172.2.0.50.80.2.20241026132721115_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp028/thyroidNodules_axp-028.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thy

Total generate 261 image...
{'datetime': '2024-12-1 21:33:59', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp034/thyroidNodules_axp-034.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp035 [23/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp035/1.2.250.1.204.5.8373724313.20210224150443460916.2.0.50.80.2.20241104143020795_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp035/1.2.250.1.204.5.8373724313.20210224150443460916.2.0.50.80.2.20241104143020795_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp035/thyroidNodules_axp-035.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFo

...

Total generate 235 image...
{'datetime': '2024-12-1 21:34:29', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp065/thyroidNodules_axp-065.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp068 [51/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp068/1.2.250.1.204.5.8373724313.20210413152441262036.2.0.50.80.2.20241109092950508_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp068/1.2.250.1.204.5.8373724313.20210413152441262036.2.0.50.80.2.20241109092950508_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp068/thyroidNodules_axp-068.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFo

Total generate 312 image...
{'datetime': '2024-12-1 21:34:37', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp075/thyroidNodules_axp-075.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp076 [58/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp076/1.2.250.1.204.5.8373724313.20210415143257934954.2.0.50.80.2.20241109211518271_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp076/1.2.250.1.204.5.8373724313.20210415143257934954.2.0.50.80.2.20241109211518271_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp076/thyroidNodules_axp-076.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFo

Total generate 265 image...
{'datetime': '2024-12-1 21:34:47', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp084/thyroidNodules_axp-084.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp085 [65/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp085/1.2.250.1.204.5.8373724313.20210416143704229580.2.0.50.80.2.20241110173525357_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp085/1.2.250.1.204.5.8373724313.20210416143704229580.2.0.50.80.2.20241110173525357_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp085/thyroidNodules_axp-085.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFo

Total generate 357 image...
{'datetime': '2024-12-1 21:34:54', 'fps': 100, 'imagepath': '/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp093/thyroidNodules_axp-093.dcm_frms', 'raw_file_type': 'dcm'}
********************************
Handle with /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp101 [72/]
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp101/1.2.250.1.204.5.667136.20211026093804621198.2.0.50.80.2.20241112143343375_MARK.json
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp101/1.2.250.1.204.5.667136.20211026093804621198.2.0.50.80.2.20241112143343375_MARK.json
file in dir: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thyroidNodules_axp101/thyroidNodules_axp-101.dcm
dcmfile: /mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat/thy

0